In [1]:
%load_ext autoreload
%autoreload
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import shapefile
import pygeohash as geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniformc
from geoUtils import *
basedir="/Users/tgadf/Downloads/osm"

Notebook Last Run Initiated: 2018-09-17 14:03:18.872471


In [4]:
from modelio import loadJoblib
buildingLookup = loadJoblib("buildingLookup.p")
buildingMap    = loadJoblib("buildingMap.p")
buildingConv   = loadJoblib("buildingConv.p")

def parseOSM(dirval):
    from os.path import basename, dirname, join, splitext
    from glob import glob
    from collections import Counter
    state = basename(dirval).replace("-latest-free.shp", "")
    print(state)
    print(dirval)
    useUnique = False
    osmpath = dirval
    osms     = [splitext(basename(x))[0] for x in glob(join(osmpath, "*build*.shp"))]
    blgCntr = Counter()
    nameCntr = Counter()
    nameLookup = {}
    for osm in osms:
        print(osm)
        shapeval = osm
        
        files = glob(join(basedir, "{0}-{1}-{2}*".format(state, osm, 7)))
        if len(files) > 0:
            print("Already processed this one...")
            #continue

        try:
            sf = shapefile.Reader(join(osmpath, osm))
        except:
            raise ValueError("No shapefile!")
        fields      = sf.fields
        shapeData   = {}
        geoShapeMap = {}
        Nshapes   = len(sf.shapes())
        ngeos     = 0
        totalgeos = 0
        prec      = 7
        show      = False
        cntr      = Counter()

        start,cmt = clock("\n\nAnalyzing {0}\t{1}".format(shapeval, Nshapes))
        if show:
            print("Fields -> {0}".format(fields))

        irec = -1
        for shapeRec in sf.iterShapeRecords():
            irec += 1
            if irec % 25000 == 0 and irec > 0:
                if True:
                    print("Processed {0}/{1} records. Found {2} geos so far...".format(irec, Nshapes, ngeos))

            ## Record
            record = shapeRec.record
            if show:
                print("Stopping at 1st record: {0}".format(record))
                break
            if osm in ["gis_osm_buildings_a_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = "building"
                fclass = record[2]
                name   = record[3]
                ftype  = record[4]
                if buildingLookup.get(name) is not None:
                    fclass = buildingLookup[name]
                    geoid  = fclass
                else:
                    if len(ftype) > 0:
                        fclass = ftype
                        geoid  = fclass
                    else:
                        found=None
                        for k,v in buildingMap.items():
                            if k in name.split():
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                            if name.find("{0} ".format(k)) != -1 or name.find(" {0}".format(k)) != -1:
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                                
                        if found is None:
                            nameCntr[name] += 1
                            if nameLookup.get(name) is None:
                                if len(ftype) > 0:
                                    nameLookup[name] = ftype
                            for val in name.split():
                                blgCntr[val] += 1
                if buildingConv.get(geoid):
                    geoid = buildingConv[geoid]
                if buildingConv.get(fclass):
                    fclass = buildingConv[fclass]
            elif osm in ["gis_osm_pofw_a_free_1", "gis_osm_pofw_free_1", "gis_osm_pois_a_free_1", "gis_osm_pois_free_1", "gis_osm_railways_free_1"] or osm in ["gis_osm_traffic_a_free_1", "gis_osm_traffic_free_1", "gis_osm_transport_a_free_1", "gis_osm_transport_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
            else:
                print("Not recording {0}".format(osm))
                break
#            elif osm in ["gis_osm_places_a_free_1", "gis_osm_places_free_1"]:
#                if useUnique:
#                    geoid  = record[0]
#                else:
#                    code   = record[1]
#                    geoid  = code
#                fclass = record[2]
#                pop    = record[3]
#                name   = record[4]

            shapeData[geoid] = {"Name": fclass, "Record": irec}
            cntr[geoid] += 1
            #continue

            record = shapeRec.record
            shape  = shapeRec.shape
            points = shape.points
            useLinear = False
            if len(points) == 1 or useLinear is True:
                useLinear = True
                geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False)
            else:
                geos = getInitGeo(shape)
                geos = addShapeGeos(irec, Nshapes, shape, prec, geos, debug=False)
                if geos is None:
                    useLinear = True
                    geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False) 

            if geoShapeMap.get(geoid) is None:
                geoShapeMap[geoid] = {}
            for geo in geos:
                geoShapeMap[geoid][geo] = True
            #geoShapeMap[geoid] = geoShapeMap[geoid].union(geos)
            ngeos += len(geos)

        #print(cntr.most_common(100))
        found = False
        for k,v in cntr.most_common():
            k = str(k)
            if k.find(" ") != -1 or k.find(";") != -1 or k.find("_") != -1 or k.find(",") != -1 or k[0].isupper():
                print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
                found = True
        print("\n")
        for k,v in nameCntr.most_common(200):
            print("commerce[\"{0}\"] = \"{1}\"".format(k, nameLookup.get(k)))
        if len(shapeData) > 0:
            for geoid in geoShapeMap.keys():
                geoShapeMap[geoid] = list(geoShapeMap[geoid].keys())
            print("Found {0} geos from {1}".format(ngeos, shapeval))
            saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, join(basedir, "{0}-{1}-{2}".format(state, osm, 7)))

In [ ]:
import zipfile
from glob import glob
from os import mkdir
from os.path import splitext, basename, dirname, join, exists
zipfiles = glob(join(basedir, "[l-n]*.zip"))
for zipname in zipfiles:
    statedir = dirname(zipname)
    name     = splitext(basename(zipname))[0]
    dirval   = join(statedir, name) 
    if exists(dirval):
        continue
    try:
        mkdir(dirval)
    except:
        pass
    zip_ref = zipfile.ZipFile(zipname, 'r')
    zip_ref.extractall(dirval)
    
    try:
        parseOSM(dirval)
    except:
        pass
    
    zip_ref.close()
    
    rmZipDir(dirval)

louisiana
/Users/tgadf/Downloads/osm/louisiana-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 15:17:03 for 

Analyzing gis_osm_buildings_a_free_1	191639
Processed 25000/191639 records. Found 33189 geos so far...
Processed 50000/191639 records. Found 64298 geos so far...
Processed 75000/191639 records. Found 95874 geos so far...
Processed 100000/191639 records. Found 128187 geos so far...
Processed 125000/191639 records. Found 158970 geos so far...
Processed 150000/191639 records. Found 190196 geos so far...
Processed 175000/191639 records. Found 221534 geos so far...


commerce[""] = "None"
commerce["English Village Commons"] = "None"
commerce["Rouses"] = "None"
commerce["H.O. West Physical Education Building"] = "None"
commerce["Brookshire Brothers"] = "None"
commerce["New Orleans Regional Transit Authority"] = "None"
commerce["Lynn Alumni and Development Center"] = "None"
commerce["House"] = "None"
commerce["Facility Services

maine
/Users/tgadf/Downloads/osm/maine-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 15:20:30 for 

Analyzing gis_osm_buildings_a_free_1	68315
Processed 25000/68315 records. Found 34171 geos so far...
Processed 50000/68315 records. Found 66029 geos so far...


commerce[""] = "None"
commerce["Hannaford"] = "None"
commerce["Big Apple"] = "None"
commerce["Kents Hill Lumber Co. Inc."] = "None"
commerce["Welcome Center"] = "None"
commerce["House"] = "None"
commerce["Amato's"] = "None"
commerce["Canada Border Services Agency"] = "None"
commerce["Bath Savings Institution"] = "None"
commerce["Clubhouse"] = "None"
commerce["Race Horse Paddocks"] = "None"
commerce["IGA"] = "None"
commerce["bunkhouse"] = "None"
commerce["storage"] = "None"
commerce["Dana"] = "None"
commerce["Pavillion"] = "None"
commerce["Emera Maine"] = "None"
commerce["McCain Foods Ltd."] = "None"
commerce["CBSA-ASFC"] = "None"
commerce["Weathervane"] = "None"
commerce

maryland
/Users/tgadf/Downloads/osm/maryland-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 15:22:05 for 

Analyzing gis_osm_buildings_a_free_1	1257150
Processed 25000/1257150 records. Found 37946 geos so far...
Processed 50000/1257150 records. Found 70551 geos so far...
Processed 75000/1257150 records. Found 103608 geos so far...
Processed 100000/1257150 records. Found 142131 geos so far...
Processed 125000/1257150 records. Found 174305 geos so far...
Processed 150000/1257150 records. Found 205348 geos so far...
Processed 175000/1257150 records. Found 236365 geos so far...
Processed 200000/1257150 records. Found 267566 geos so far...
Processed 225000/1257150 records. Found 298511 geos so far...
Processed 250000/1257150 records. Found 328641 geos so far...
Processed 275000/1257150 records. Found 361013 geos so far...
Processed 300000/1257150 records. Found 389383 geos so far...
Processed 325000/1257150 records. Found 419108 geo

Saved shape data to /Users/tgadf/Downloads/osm/maryland-gis_osm_buildings_a_free_1-7-data.p


There are 1537952 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/maryland-gis_osm_buildings_a_free_1-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/maryland-latest-free.shp
massachusetts
/Users/tgadf/Downloads/osm/massachusetts-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 15:44:46 for 

Analyzing gis_osm_buildings_a_free_1	2607816
Processed 25000/2607816 records. Found 31571 geos so far...
Processed 50000/2607816 records. Found 62956 geos so far...
Processed 75000/2607816 records. Found 94351 geos so far...
Processed 100000/2607816 records. Found 125856 geos so far...
Processed 125000/2607816 records. Found 157239 geos so far...
Processed 150000/2607816 records. Found 188674 geos so far...
Processed 175000/2607816 records. Found 219982 geos so far...
Processed 200000/2607816 records. Found 251397 g

Saved shape data to /Users/tgadf/Downloads/osm/massachusetts-gis_osm_buildings_a_free_1-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/massachusetts-latest-free.shp
michigan
/Users/tgadf/Downloads/osm/michigan-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 16:31:06 for 

Analyzing gis_osm_buildings_a_free_1	231690
Processed 25000/231690 records. Found 39951 geos so far...
Processed 50000/231690 records. Found 76361 geos so far...
Processed 75000/231690 records. Found 111661 geos so far...
Processed 100000/231690 records. Found 144661 geos so far...
Processed 125000/231690 records. Found 176834 geos so far...
Processed 150000/231690 records. Found 210655 geos so far...
Processed 175000/231690 records. Found 244324 geos so far...
Processed 200000/231690 records. Found 277973 geos so far...
Processed 225000/231690 records. Found 310390 geos so far...


commerce[""] = "None"
commerce["home"] = "None"
commerce["Hangar"] = "

...

Processed 50000/287364 records. Found 73314 geos so far...
Processed 75000/287364 records. Found 110517 geos so far...
Processed 100000/287364 records. Found 142803 geos so far...
Processed 125000/287364 records. Found 174641 geos so far...
Processed 150000/287364 records. Found 207097 geos so far...
Processed 175000/287364 records. Found 239042 geos so far...
Processed 200000/287364 records. Found 272667 geos so far...
Processed 225000/287364 records. Found 307251 geos so far...
Processed 250000/287364 records. Found 339651 geos so far...
Processed 275000/287364 records. Found 372991 geos so far...


commerce[""] = "None"
commerce["QuickChek"] = "None"
commerce["Lukoil"] = "None"
commerce["Delta"] = "None"
commerce["Nelson Biology Laboratories"] = "None"
commerce["Greeter’s Booth"] = "None"
commerce["United Airlines"] = "None"
commerce["American Legion Hall"] = "None"
commerce["Heritage's"] = "None"
commerce["Santander"] = "None"
commerce["Acme"] = "None"
commerce["Park Office"] = "No

new-mexico
/Users/tgadf/Downloads/osm/new-mexico-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 17:00:51 for 

Analyzing gis_osm_buildings_a_free_1	112823
Processed 25000/112823 records. Found 35244 geos so far...
Processed 50000/112823 records. Found 67166 geos so far...
Processed 75000/112823 records. Found 99292 geos so far...
Processed 100000/112823 records. Found 131322 geos so far...


commerce[""] = "None"
commerce["Manzano Mountain Retreat"] = "None"
commerce["Pic Quik"] = "None"
commerce["Alon"] = "None"
commerce["Pecos River Cabins"] = "None"
commerce["Mattox Sculpture Center"] = "None"
commerce["Green Jeans Farmery"] = "None"
commerce["Dion's"] = "None"
commerce["Burger Time"] = "None"
commerce["Twisters"] = "None"
commerce["AmeriGas"] = "None"
commerce["Fina"] = "None"
commerce["Golden Pride"] = "None"
commerce["Professional Building"] = "None"
commerce["Sadie's of New Mexico"] = "None"
commerce["Big 5"] = "None"
co

new-york
/Users/tgadf/Downloads/osm/new-york-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 17:03:37 for 

Analyzing gis_osm_buildings_a_free_1	2042437
Processed 25000/2042437 records. Found 41337 geos so far...
Processed 50000/2042437 records. Found 72832 geos so far...
Processed 75000/2042437 records. Found 103543 geos so far...
Processed 100000/2042437 records. Found 133615 geos so far...
Processed 125000/2042437 records. Found 163204 geos so far...
Processed 150000/2042437 records. Found 193594 geos so far...
Processed 175000/2042437 records. Found 223741 geos so far...
Processed 200000/2042437 records. Found 253609 geos so far...
Processed 225000/2042437 records. Found 284173 geos so far...
Processed 250000/2042437 records. Found 314957 geos so far...
Processed 275000/2042437 records. Found 345383 geos so far...
Processed 300000/2042437 records. Found 375377 geos so far...
Processed 325000/2042437 records. Found 405669 geo

north-carolina
/Users/tgadf/Downloads/osm/north-carolina-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 17:39:43 for 

Analyzing gis_osm_buildings_a_free_1	537764
Processed 25000/537764 records. Found 32150 geos so far...
Processed 50000/537764 records. Found 63362 geos so far...
Processed 75000/537764 records. Found 94335 geos so far...
Processed 100000/537764 records. Found 124858 geos so far...
Processed 125000/537764 records. Found 156282 geos so far...
Processed 150000/537764 records. Found 187029 geos so far...
Processed 175000/537764 records. Found 218628 geos so far...
Processed 200000/537764 records. Found 251429 geos so far...
Processed 225000/537764 records. Found 283312 geos so far...
Processed 250000/537764 records. Found 318965 geos so far...
Processed 275000/537764 records. Found 350604 geos so far...
Processed 300000/537764 records. Found 380988 geos so far...
Processed 325000/537764 records. Found 412620 geos s

north-dakota
/Users/tgadf/Downloads/osm/north-dakota-latest-free.shp
gis_osm_buildings_a_free_1
Already processed this one...
Current Time is Mon Sep 17, 2018 17:49:08 for 

Analyzing gis_osm_buildings_a_free_1	151896
Processed 25000/151896 records. Found 31389 geos so far...
Processed 50000/151896 records. Found 63511 geos so far...
Processed 75000/151896 records. Found 95060 geos so far...
Processed 100000/151896 records. Found 125983 geos so far...
Processed 125000/151896 records. Found 156222 geos so far...
Processed 150000/151896 records. Found 187068 geos so far...


commerce[""] = "None"
commerce["EERC"] = "None"
commerce["Tesoro"] = "None"
commerce["Blockhouse"] = "None"
commerce["State Farm Insurance"] = "None"
commerce["Dakota Hall"] = "None"
commerce["Waldron Hall"] = "None"
commerce["Loaf 'N Jug"] = "None"
commerce["250"] = "None"
commerce["Mexican Village"] = "None"
commerce["Concessions"] = "None"
commerce["Hornbacher's"] = "None"
commerce["Hoiberg's Office Supply and Pri